In [ ]:
# === KAGGLE SUBMISSION WITH ADVANCED FEATURES ===
import os
import pandas as pd
import polars as pl
import numpy as np
from pathlib import Path
import json
import pickle

# Import required libraries from your existing notebook
from scipy.stats import spearmanr
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import kaggle_evaluation.mitsui_inference_server

# Configuration constants
NUM_TARGET_COLUMNS = 424

# Auto-detect environment and set correct paths
def get_data_path():
    """Detect environment and return appropriate data path"""
    kaggle_path = Path('/kaggle/input/mitsui-commodity-prediction-challenge')
    local_path = Path("dataset")
    
    if kaggle_path.exists():
        print("🔧 Kaggle environment detected")
        return kaggle_path
    else:
        print("🔧 Local development environment detected")
        return local_path

data_path_MITSUI = get_data_path()

# === YOUR ORIGINAL create_advanced_features() FUNCTION ===
def create_advanced_features(ml_dataset_filtered, feature_cols_only):
    """
    Create comprehensive technical and statistical features for ensemble training.
    
    Args:
        ml_dataset_filtered: DataFrame with filtered ML data
        feature_cols_only: List of feature columns to process
    
    Returns:
        ml_dataset_filtered: DataFrame with added features
    """
    print("🔧 CREATING ADVANCED FEATURES FOR ENSEMBLE TRAINING")
    print("=" * 50)
    
    feature_count_before = len(ml_dataset_filtered.columns)
    
    # Add rolling statistics - all columns should now be numeric
    for col in feature_cols_only:
        if col[0] != '_':  # Skip date_id and target
            try:
                # Rolling mean (3, 5, 10, 20 periods)
                ml_dataset_filtered[('_', f'rolling_mean_3_{col[1]}')] = ml_dataset_filtered[col].rolling(window=3).mean()
                ml_dataset_filtered[('_', f'rolling_mean_5_{col[1]}')] = ml_dataset_filtered[col].rolling(window=5).mean()
                ml_dataset_filtered[('_', f'rolling_mean_10_{col[1]}')] = ml_dataset_filtered[col].rolling(window=10).mean()
                ml_dataset_filtered[('_', f'rolling_mean_20_{col[1]}')] = ml_dataset_filtered[col].rolling(window=20).mean()
                
                # Rolling std (3, 5, 10, 20 periods) → Volatilità locale
                ml_dataset_filtered[('_', f'rolling_std_3_{col[1]}')] = ml_dataset_filtered[col].rolling(window=3).std()
                ml_dataset_filtered[('_', f'rolling_std_5_{col[1]}')] = ml_dataset_filtered[col].rolling(window=5).std()
                ml_dataset_filtered[('_', f'rolling_std_10_{col[1]}')] = ml_dataset_filtered[col].rolling(window=10).std()
                ml_dataset_filtered[('_', f'rolling_std_20_{col[1]}')] = ml_dataset_filtered[col].rolling(window=20).std()
                
                # Annualized volatility (√252)
                ml_dataset_filtered[('_', f'annual_vol_20_{col[1]}')] = (
                    ml_dataset_filtered[col].rolling(window=20).std() * np.sqrt(252)
                )
                
                # Percentage change
                ml_dataset_filtered[('_', f'pct_change_{col[1]}')] = ml_dataset_filtered[col].pct_change()
                
                # Lag features (1, 2, 3 periods)
                ml_dataset_filtered[('_', f'lag_1_{col[1]}')] = ml_dataset_filtered[col].shift(1)
                ml_dataset_filtered[('_', f'lag_2_{col[1]}')] = ml_dataset_filtered[col].shift(2)
                ml_dataset_filtered[('_', f'lag_3_{col[1]}')] = ml_dataset_filtered[col].shift(3)
                
                # Skewness (10, 20 giorni)
                ml_dataset_filtered[('_', f'rolling_skew_10_{col[1]}')] = ml_dataset_filtered[col].rolling(window=10).skew()
                ml_dataset_filtered[('_', f'rolling_skew_20_{col[1]}')] = ml_dataset_filtered[col].rolling(window=20).skew()
                
                # Kurtosis (10, 20 giorni)
                ml_dataset_filtered[('_', f'rolling_kurt_10_{col[1]}')] = ml_dataset_filtered[col].rolling(window=10).kurt()
                ml_dataset_filtered[('_', f'rolling_kurt_20_{col[1]}')] = ml_dataset_filtered[col].rolling(window=20).kurt()
                
                # Autocorrelazione (lag 1, 5)
                ml_dataset_filtered[('_', f'autocorr_1_{col[1]}')] = ml_dataset_filtered[col].rolling(window=20).apply(
                    lambda x: x.autocorr(lag=1), raw=False
                )
                ml_dataset_filtered[('_', f'autocorr_5_{col[1]}')] = ml_dataset_filtered[col].rolling(window=20).apply(
                    lambda x: x.autocorr(lag=5), raw=False
                )
                
                # Volatility-of-volatility (Vol-of-Vol)
                rolling_vol = ml_dataset_filtered[col].rolling(window=10).std()
                ml_dataset_filtered[('_', f'vol_of_vol_10_{col[1]}')] = rolling_vol.rolling(window=10).std()
                
                # Regime features (binari)
                rolling_mean_10 = ml_dataset_filtered[col].rolling(window=10).mean()
                rolling_vol_10 = ml_dataset_filtered[col].rolling(window=10).std()
                
                ml_dataset_filtered[('_', f'regime_trend_up_{col[1]}')] = (rolling_mean_10 > 0).astype(int)
                ml_dataset_filtered[('_', f'regime_high_vol_{col[1]}')] = (
                    rolling_vol_10 > rolling_vol_10.quantile(0.75)
                ).astype(int)

            except Exception as e:
                print(f"⚠️  Error processing column {col}: {e}")
    
    feature_count_after = len(ml_dataset_filtered.columns)
    features_added = feature_count_after - feature_count_before
    
    print(f"✅ FEATURE ENGINEERING COMPLETED")
    print(f"   Features before: {feature_count_before}")
    print(f"   Features after: {feature_count_after}")
    print(f"   Features added: {features_added}")
    
    return ml_dataset_filtered

# === MULTIINDEX PROCESSING FUNCTIONS ===
def get_category(col):
    """Extract category from column name based on prefix"""
    if isinstance(col, tuple):
        col = col[1]  # Take the feature name from tuple (category, feature)
    
    col = str(col).lower().strip()
    
    if col.startswith("us_stock_"):
        return "us"
    elif col.startswith("jpx_"):
        return "jpx"
    elif col.startswith("fx_"):
        return "fx"
    elif col.startswith("lme_"):
        return "lme"
    else:
        return "other"

def get_instrument(col):
    """Extract instrument name from column name"""
    if isinstance(col, tuple):
        col = col[1]
    
    col = str(col).lower().strip()
    parts = col.split("_")
    
    if col.startswith("us_stock_") and len(parts) >= 3:
        return parts[2]
    elif col.startswith("fx_") and len(parts) >= 2:
        return parts[1]
    elif col.startswith("jpx_") and len(parts) >= 2:
        return parts[1]
    elif col.startswith("lme_") and len(parts) >= 2:
        return parts[1]
    else:
        return str(col)

# === TRAINED MODELS STORAGE WITH ADVANCED FEATURES ===
class ModelStorage:
    """Global storage for trained models with advanced feature engineering"""
    def __init__(self):
        self.models = {}
        self.feature_columns = {}
        self.scaler = None
        self.target_pairs = None
        self.is_initialized = False
        
    def initialize_models(self):
        """Initialize models with advanced feature engineering during first predict call"""
        if self.is_initialized:
            return
            
        print("🔧 INITIALIZING MODELS WITH ADVANCED FEATURES FOR KAGGLE SUBMISSION...")
        
        try:
            # Load target pairs information
            target_pairs_path = data_path_MITSUI / "target_pairs.csv"
            print(f"   📂 Loading target pairs from: {target_pairs_path}")
            self.target_pairs = pd.read_csv(target_pairs_path)
            self.target_pairs.columns = self.target_pairs.columns.str.lower().str.strip()
            
            # Debug: Show available columns in target_pairs
            print(f"   🔍 Target pairs columns: {list(self.target_pairs.columns)}")
            print(f"   📊 Target pairs shape: {self.target_pairs.shape}")
            if len(self.target_pairs) > 0:
                print(f"   🎯 Sample target: {self.target_pairs.iloc[0].to_dict()}")
            
            # Load and prepare training data
            print("   📊 Loading training data...")
            train_path = data_path_MITSUI / 'train.csv'
            train_labels_path = data_path_MITSUI / 'train_labels.csv'
            
            df_train_fast = pd.read_csv(train_path, dtype={'date_id': 'int64'})
            df_train_labels_fast = pd.read_csv(train_labels_path, dtype={'date_id': 'int64'})
            
            # Clean column names
            df_train_fast.columns = df_train_fast.columns.str.lower().str.strip()
            df_train_labels_fast.columns = df_train_labels_fast.columns.str.lower().str.strip()
            
            # Basic data cleaning (fixed deprecated fillna method)
            df_train_fast = df_train_fast.ffill().fillna(0)
            df_train_labels_fast = df_train_labels_fast.ffill().fillna(0)
            
            # CREATE MULTIINDEX STRUCTURE FOR ADVANCED FEATURES
            print("   🔧 Creating MultiIndex structure for advanced feature engineering...")
            
            # Get feature columns (exclude date_id)
            feature_cols = [col for col in df_train_fast.columns if col != 'date_id']
            
            # Extract categories for MultiIndex
            categories = [get_category(c) for c in feature_cols]
            
            # Create MultiIndex for ALL columns (including date_id)
            all_columns = ['date_id'] + feature_cols
            all_categories = ['_'] + categories
            
            multi_index = pd.MultiIndex.from_arrays(
                [all_categories, all_columns],
                names=["category", "feature"]
            )
            
            # Apply MultiIndex to training data
            df_train_fast.columns = multi_index
            
            # Restore date_id as integer
            date_id_col = ('_', 'date_id')
            df_train_fast[date_id_col] = df_train_fast[date_id_col].astype('int64')
            
            print(f"   ✅ MultiIndex applied with {len(multi_index)} columns")
            
            # Train models for priority targets with ADVANCED FEATURES
            # Check if 'target' column exists, otherwise use first column or index
            if 'target' in self.target_pairs.columns:
                top_targets = self.target_pairs['target'].head(5).tolist()
            elif len(self.target_pairs.columns) > 0:
                # Use first column as target names
                first_col = self.target_pairs.columns[0]
                top_targets = self.target_pairs[first_col].head(5).tolist()
                print(f"   ⚠️ No 'target' column found, using '{first_col}' column instead")
            else:
                # Fallback to generated target names
                top_targets = [f'target_{i}' for i in range(5)]
                print(f"   ⚠️ No target columns found, using generated target names")
            
            print(f"   🏗️ Training models with advanced features for {len(top_targets)} priority targets...")
            print(f"   🎯 Target list: {top_targets[:3]}..." + (f" (+{len(top_targets)-3} more)" if len(top_targets) > 3 else ""))
            
            for i, target in enumerate(top_targets):
                if target in df_train_labels_fast.columns:
                    print(f"      🎯 Training {target} with advanced features ({i+1}/{len(top_targets)})...")
                    
                    try:
                        # Get target information for category-based feature selection
                        target_info = self.target_pairs[self.target_pairs['target'] == target] if 'target' in self.target_pairs.columns else []
                        
                        # Handle missing 'category' column gracefully
                        if 'category' in self.target_pairs.columns and len(target_info) > 0:
                            category = target_info['category'].values[0]
                        else:
                            # Fallback: extract category from target name or use 'other'
                            if target.startswith('fx_'):
                                category = 'fx'
                            elif target.startswith('us_'):
                                category = 'us'
                            elif target.startswith('jpx_'):
                                category = 'jpx'
                            elif target.startswith('lme_'):
                                category = 'lme'
                            else:
                                category = 'fx'  # Default to fx as most common
                        
                        print(f"         🔧 Using category '{category}' for target {target}")
                        
                        # Select relevant features based on target category
                        relevant_columns = []
                        
                        # Add utility columns
                        relevant_columns.append(date_id_col)
                        
                        # Add primary category features
                        if category in df_train_fast.columns.get_level_values(0):
                            primary_features = [col for col in df_train_fast.columns if col[0] == category]
                            relevant_columns.extend(primary_features[:20])  # Limit for speed
                        
                        # Add some features from other categories for diversity
                        other_categories = ['fx', 'lme', 'us', 'jpx']
                        for other_cat in other_categories:
                            if other_cat != category and other_cat in df_train_fast.columns.get_level_values(0):
                                other_features = [col for col in df_train_fast.columns if col[0] == other_cat][:5]
                                relevant_columns.extend(other_features)
                        
                        # Remove duplicates
                        relevant_columns = list(dict.fromkeys(relevant_columns))
                        
                        # Filter dataset to relevant features
                        ml_dataset_filtered = df_train_fast[relevant_columns].copy()
                        
                        # Get feature columns for advanced feature engineering (exclude date_id)
                        feature_cols_only = [col for col in relevant_columns if col != date_id_col]
                        
                        print(f"         🔧 Applying advanced feature engineering to {len(feature_cols_only)} base features...")
                        
                        # APPLY YOUR ADVANCED FEATURE ENGINEERING
                        ml_dataset_filtered = create_advanced_features(ml_dataset_filtered, feature_cols_only)
                        
                        # Add target data
                        y = df_train_labels_fast[target]
                        
                        # Align data lengths
                        min_len = min(len(ml_dataset_filtered), len(y))
                        X_dataset = ml_dataset_filtered.iloc[-min_len:].copy()
                        y_target = y.iloc[-min_len:].copy()
                        
                        # Clean up data after feature engineering
                        X_dataset = X_dataset.dropna()
                        
                        # Align target with cleaned features
                        y_target = y_target.loc[X_dataset.index]
                        
                        if len(X_dataset) > 100:  # Minimum samples
                            # Flatten column names for sklearn
                            new_columns = []
                            for col in X_dataset.columns:
                                if isinstance(col, tuple):
                                    new_columns.append(f"{col[0]}_{col[1]}")
                                else:
                                    new_columns.append(str(col))
                            
                            X_dataset.columns = new_columns
                            
                            # Remove date_id column for training
                            if '_date_id' in X_dataset.columns:
                                X_clean = X_dataset.drop(columns=['_date_id'])
                            else:
                                X_clean = X_dataset
                            
                            print(f"         📊 Final training data: {X_clean.shape} features, {len(y_target)} samples")
                            
                            # Use ensemble model (your best performing method)
                            try:
                                model = StackingRegressor([
                                    ('lr', LinearRegression()),
                                    ('lgb', self.create_fast_lightgbm())
                                ], final_estimator=LinearRegression(), cv=3)
                                
                                model.fit(X_clean, y_target)
                                self.models[target] = model
                                self.feature_columns[target] = X_clean.columns.tolist()
                                
                                print(f"         ✅ Stacking ensemble trained with {len(X_clean.columns)} advanced features")
                                
                            except Exception as e:
                                print(f"         ⚠️ Stacking failed, using VotingRegressor: {e}")
                                model = VotingRegressor([
                                    ('lr', LinearRegression()),
                                    ('lgb', self.create_fast_lightgbm())
                                ])
                                model.fit(X_clean, y_target)
                                self.models[target] = model
                                self.feature_columns[target] = X_clean.columns.tolist()
                        
                        else:
                            print(f"         ⚠️ Insufficient samples after advanced feature engineering: {len(X_dataset)}")
                            
                    except Exception as e:
                        print(f"         ❌ Error training {target} with advanced features: {e}")
            
            # Create default model
            if len(self.models) > 0:
                default_model_key = list(self.models.keys())[0]
                self.default_model = self.models[default_model_key]
                self.default_features = self.feature_columns[default_model_key]
                
                print(f"   ✅ Trained {len(self.models)} models with advanced features")
                print(f"   🔧 Default model set: {default_model_key}")
                print(f"   📊 Average features per model: {np.mean([len(self.feature_columns[k]) for k in self.feature_columns]):.0f}")
            else:
                # Ultimate fallback
                print("   ⚠️ Creating ultimate fallback model...")
                feature_cols_basic = [col for col in df_train_fast.columns if col != date_id_col][:20]
                X_basic = df_train_fast[feature_cols_basic].fillna(0)
                
                # Flatten columns for sklearn
                new_columns = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) else str(col) for col in X_basic.columns]
                X_basic.columns = new_columns
                
                self.default_model = LinearRegression()
                self.default_model.fit(X_basic, df_train_labels_fast.iloc[:, 1].fillna(0))
                self.default_features = X_basic.columns.tolist()
            
            self.is_initialized = True
            print("   🎉 ADVANCED FEATURE MODEL INITIALIZATION COMPLETED!")
            
        except Exception as e:
            print(f"   ❌ Advanced feature model initialization failed: {e}")
            import traceback
            traceback.print_exc()
            # Create minimal fallback
            self.default_model = LinearRegression()
            self.default_features = []
            self.is_initialized = True
    
    def create_fast_lightgbm(self):
        """Create fast LightGBM for Kaggle submission"""
        return lgb.LGBMRegressor(
            n_estimators=50,  # Reduced for speed
            max_depth=6,
            learning_rate=0.1,
            random_state=42,
            verbose=-1,
            n_jobs=1,
            force_row_wise=True
        )

# Global model storage
model_storage = ModelStorage()

def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pl.DataFrame:
    """
    Kaggle prediction function with advanced features
    """
    # Initialize models on first call (within time limit)
    if not model_storage.is_initialized:
        model_storage.initialize_models()
    
    # Convert to pandas for compatibility
    test_df = test.to_pandas()
    
    # Clean column names to match training
    test_df.columns = test_df.columns.str.lower().str.strip()
    
    # Initialize predictions dictionary
    predictions_dict = {}
    
    # Generate predictions for each target
    for i in range(NUM_TARGET_COLUMNS):
        target_name = f'target_{i}'
        
        try:
            if target_name in model_storage.models:
                # Use trained model with advanced features
                model = model_storage.models[target_name]
                features = model_storage.feature_columns[target_name]
                
                # Prepare features (handle missing columns gracefully)
                available_features = [f for f in features if f in test_df.columns]
                
                if available_features:
                    X_test = test_df[available_features].fillna(0)
                    pred = model.predict(X_test)[0]  # Single prediction
                else:
                    pred = 0.0  # Fallback
            
            elif hasattr(model_storage, 'default_model'):
                # Use default model
                available_features = [f for f in model_storage.default_features if f in test_df.columns]
                
                if available_features:
                    X_test = test_df[available_features].fillna(0)
                    pred = model_storage.default_model.predict(X_test)[0]
                else:
                    pred = 0.0
            else:
                pred = 0.0  # Ultimate fallback
                
        except Exception as e:
            # Silent fallback for production
            pred = 0.0
        
        predictions_dict[target_name] = pred
    
    # Create predictions DataFrame in required format
    predictions = pl.DataFrame(predictions_dict)
    
    # Verify format requirements
    assert isinstance(predictions, pl.DataFrame)
    assert len(predictions) == 1
    assert len(predictions.columns) == NUM_TARGET_COLUMNS
    
    return predictions

# === KAGGLE INFERENCE SERVER SETUP ===
print("🚀 SETTING UP KAGGLE INFERENCE SERVER WITH ADVANCED FEATURES...")

# Create the inference server
inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

# Check if running in Kaggle competition environment
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("🏆 RUNNING IN KAGGLE COMPETITION MODE")
    print("   🔧 Starting inference server...")
    inference_server.serve()

else:
    print("🧪 RUNNING IN LOCAL DEVELOPMENT MODE")
    print("   🔧 Starting local gateway for testing...")
    
    # Use the detected data path for local gateway
    input_path = str(data_path_MITSUI)
    print(f"   📂 Using data path: {input_path}")
    
    try:
        inference_server.run_local_gateway((input_path,))
    except Exception as e:
        print(f"⚠️ Local gateway failed: {e}")
        print("💡 This is normal if running outside Kaggle environment")

print("✅ KAGGLE SUBMISSION WITH ADVANCED FEATURES SETUP COMPLETED!")
print("🔧 Features included: Rolling stats, volatility, lags, skewness, kurtosis, autocorrelations, regime features")
print("🏆 Using ensemble models (Stacking/VotingRegressor) trained on comprehensive feature set")